In [ ]:
import requests
import json
import time

def fetch_reviews(max_pages=None, delay=1):
    """
    Функция для сбора отзывов с сайта banki.ru по заданному банку и сохранения их в JSON-файл.

    :param bank_code: Код банка, например 'promsvyazbank'
    :param max_pages: Максимальное количество страниц для парсинга (по умолчанию None - парсить все)
    :param delay: Задержка между запросами в секундах (по умолчанию 1)
    :return: None
    """
    base_url = "https://www.banki.ru/services/responses/list/ajax/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/112.0.0.0 Safari/537.36"
    }
    
    page = 1
    all_reviews = []
    has_more_pages = True
    
    while has_more_pages:
        params = {
            "page": page,
            "is_countable": "on",
        }
        
        print(f"Парсинг страницы {page}...")
        response = requests.get(base_url, params=params, headers=headers)
        
        if response.status_code != 200:
            print(f"Ошибка при запросе страницы {page}: статус {response.status_code}")
            break
        
        try:
            data = response.json()
        except ValueError:
            print(f"Не удалось декодировать JSON на странице {page}")
            break
        
        reviews = data.get("data", [])
        if not reviews:
            print("Больше отзывов не найдено.")
            break
        
        for review in reviews:
            review_data = {
                "id": review.get("id"),
                "title": review.get("title"),
                "userName": review.get("userName"),
                "text": review.get("text"),
                "grade": review.get("grade"),
                "commentCount": review.get("commentCount"),
                "isCountable": review.get("isCountable"),
                "dateCreate": review.get("dateCreate"),
                "resolutionIsApproved": review.get("resolutionIsApproved"),
                "agentAnswerText": review.get("agentAnswerText"),
                "agentId": review.get("agentId"),
                "company": {
                    "id": review.get("company", {}).get("id"),
                    "code": review.get("company", {}).get("code"),
                    "name": review.get("company", {}).get("name"),
                    "url": review.get("company", {}).get("url"),
                    "logo": review.get("company", {}).get("logo"),
                    "squareLogo": review.get("company", {}).get("squareLogo"),
                    "phone": review.get("company", {}).get("phone"),
                    "region": review.get("company", {}).get("region"),
                    "address": review.get("company", {}).get("address")
                },
                "hasDocuments": review.get("hasDocuments")
            }
            all_reviews.append(review_data)
        
        has_more_pages = data.get("hasMorePages", False)
        
        if not has_more_pages:
            print("Достигнут конец списка отзывов.")
            break
        
        if max_pages and page >= max_pages:
            print(f"Достигнуто максимальное количество страниц: {max_pages}")
            break
        
        page += 1
        time.sleep(delay)  # Задержка между запросами для избежания блокировки
    
    # Сохранение в JSON
    output_data = {
        "data": all_reviews,
        "hasMorePages": has_more_pages
    }
    
    with open("banki_reviews.json", "w", encoding='utf-8') as json_file:
        json.dump(output_data, json_file, ensure_ascii=False, indent=4)
    
    print("Сбор данных завершён. Файл 'banki_reviews.json' сохранён.")

if __name__ == "__main__":
    max_pages_to_fetch = None  # Укажите количество страниц, которое хотите парсить, или None для всех
    fetch_reviews(max_pages=max_pages_to_fetch, delay=1)
